In [1]:
# 4.15 버전에서 가능한 코드

!pip install transformers==4.15.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.2 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=090bd71531538fb8021ec2c4e1627f8c2c9f109b6893a6fb6a3fe663cf6d16ac
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [2]:
import transformers
transformers.__version__

'4.15.0'

In [3]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [102]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7fb1b4f42b20>)

In [103]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [104]:
train_data

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [121]:
data = pd.read_csv('/content/drive/MyDrive/Deep_Learning/NLP/finance_sentiment_corpus-main/finance_data.csv',encoding='utf-8-sig')

data.head()
# negative 0, neutral 1, positive 2,

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [122]:
data = data.query('labels != "neutral"')

In [123]:
def sentiment_analysis(data):
  if data == 'negative':
    return 0
  elif data == 'neutral':
    return 2
  else:
    return 1

In [124]:
data['labels_tr'] = data['labels'].apply(lambda x: sentiment_analysis(x))

In [125]:
data.isnull().sum()

labels          0
sentence        0
kor_sentence    0
labels_tr       0
dtype: int64

In [126]:
data = data[['kor_sentence','labels_tr']]

In [127]:
data['kor_sentence'][5]

'ASPOCOMP의 성장기에 대한 자금 조달은 기술적으로 더 까다로운 HDI 인쇄 회로 기판 PCB에 점점 더 초점을 맞추면서 성장 전략을 공격적으로 추진하고 있다.'

In [128]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='', eos_token='', pad_token='')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [129]:
print(tokenizer.encode(data['kor_sentence'][5]))
print(tokenizer.tokenize(data['kor_sentence'][5]))

[9772, 32904, 421, 21591, 419, 422, 8143, 10622, 9399, 9167, 13462, 16992, 8135, 9698, 9115, 9267, 36980, 9520, 10553, 410, 415, 12037, 22333, 9032, 8618, 20078, 408, 8022, 12687, 9267, 22328, 17550, 9149, 10622, 29631, 10506, 9115, 26950, 10960]
['▁A', 'SP', 'O', 'CO', 'M', 'P', '의', '▁성장', '기에', '▁대한', '▁자금', '▁조달', '은', '▁기술', '적으로', '▁더', '▁까다', '로운', '▁H', 'D', 'I', '▁인쇄', '▁회로', '▁기', '판', '▁PC', 'B', '에', '▁점점', '▁더', '▁초점을', '▁맞추', '면서', '▁성장', '▁전략을', '▁공격', '적으로', '▁추진하고', '▁있다.']


In [130]:
max_seq_len = 128

In [131]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using pad_token, but it is not set yet.


In [132]:
encoded_result = tokenizer.encode(data['kor_sentence'][5],
                                  max_length=max_seq_len,
                                  pad_to_max_length=True)
print(encoded_result)
print('길이 :', len(encoded_result))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[9772, 32904, 421, 21591, 419, 422, 8143, 10622, 9399, 9167, 13462, 16992, 8135, 9698, 9115, 9267, 36980, 9520, 10553, 410, 415, 12037, 22333, 9032, 8618, 20078, 408, 8022, 12687, 9267, 22328, 17550, 9149, 10622, 29631, 10506, 9115, 26950, 10960, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201, 51201]
길이 : 128


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2226: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [133]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, data_labels = [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):

        bos_token = [tokenizer.bos_token]
        eos_token = [tokenizer.eos_token]
        tokens = bos_token + tokenizer.tokenize(example) + eos_token
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return input_ids, data_labels

In [134]:
from sklearn.model_selection import train_test_split


train, test = train_test_split(data, test_size = 0.2, random_state = 0)

In [135]:
train_X, train_y = convert_examples_to_features(train['kor_sentence'], train['labels_tr'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 1573/1573 [00:02<00:00, 732.62it/s]


In [136]:
test_X, test_y = convert_examples_to_features(test['kor_sentence'], test['labels_tr'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 394/394 [00:00<00:00, 501.31it/s]


In [137]:
# 최대 길이: 128
input_id = train_X[0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)

단어에 대한 정수 인코딩 : [51200 13365 10189 19468  9571 30512 12239 13957 13984 19201 11981 40713
  9776   439 11324   463   448 10553   411   418  9223 11981   407 39405
   425  7162 13365 20989  6958  9470  7827 15627  9034  7109 19584 48475
   451  9023  9130 11452 17258 10601 14547 10070 28296  9016 51200 51201
 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201
 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201
 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201
 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201
 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201
 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201 51201
 51201 51201 51201 51201 51201 51201 51201 51201]
각 인코딩의 길이 : 128
정수 인코딩 복원 : <|endoftext|> 2010년 8월 11일 - 핀란드 측정 장비 제조업체인 Vaisala Oyj HEL : VAIAS는 2010년 상반기 순손실이 전년 동기 2.3m에서 4.8m로 확대되었다고 오늘 밝혔다.<|endoftext|>[PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]

In [138]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.6.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.8.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [139]:
max_seq_len = 128

In [140]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer])

In [141]:
print(outputs)

TFBaseModelOutputWithPastAndCrossAttentions(last_hidden_state=<KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'tfgpt2_model_7')>, past_key_values=(<KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by lay

In [142]:
print(outputs[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 128, 768), dtype=tf.float32, name=None), name='tfgpt2_model_7/transformer/Reshape_2:0', description="created by layer 'tfgpt2_model_7'")


In [143]:
print(outputs[1])

(<KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model_7')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=

In [144]:
print(outputs[0][:, -1])

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf.__operators__.getitem_2/strided_slice:0', description="created by layer 'tf.__operators__.getitem_2'")


In [145]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.classifier = tf.keras.layers.Dense(1,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='sigmoid',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        cls_token = self.dropout(cls_token)
        prediction = self.classifier(cls_token)

        return prediction

In [146]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [147]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [151]:
with strategy.scope():
  model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.BinaryCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.6.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.8.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [152]:
model.fit(train_X, train_y, epochs=10, batch_size=16, validation_split=0.2)

Epoch 1/10


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None,) dtype=int32>]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None,) dtype=int32>]


79/79 [==============================] - ETA: 0s - loss: 1.0644 - accuracy: 0.5827

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None,) dtype=int32>]


79/79 [==============================] - 95s 185ms/step - loss: 1.0644 - accuracy: 0.5827 - val_loss: 0.5499 - val_accuracy: 0.7333
Epoch 2/10
79/79 [==============================] - 7s 88ms/step - loss: 0.4109 - accuracy: 0.8148 - val_loss: 0.1929 - val_accuracy: 0.9206
Epoch 3/10
79/79 [==============================] - 7s 83ms/step - loss: 0.1836 - accuracy: 0.9237 - val_loss: 0.2247 - val_accuracy: 0.9270
Epoch 4/10
79/79 [==============================] - 7s 94ms/step - loss: 0.1162 - accuracy: 0.9571 - val_loss: 0.2292 - val_accuracy: 0.9238
Epoch 5/10
79/79 [==============================] - 7s 90ms/step - loss: 0.0775 - accuracy: 0.9738 - val_loss: 0.1591 - val_accuracy: 0.9302
Epoch 6/10
79/79 [==============================] - 7s 91ms/step - loss: 0.0649 - accuracy: 0.9801 - val_loss: 0.1717 - val_accuracy: 0.9524
Epoch 7/10
79/79 [==============================] - 7s 92ms/step - loss: 0.0424 - accuracy: 0.9833 - val_loss: 0.3665 - val_accuracy: 0.9206
Epoch 8/10
79/79 [====

In [154]:
results = model.evaluate(test_X, test_y, batch_size=1024)
print("test loss, test acc: ", results)

1/1 [==============================] - 1s 799ms/step - loss: 0.4465 - accuracy: 0.9137
test loss, test acc:  [0.44652724266052246, 0.913705587387085]


In [155]:
def sentiment_predict(new_sentence):

  bos_token = [tokenizer.bos_token]
  eos_token = [tokenizer.eos_token]
  tokens = bos_token + tokenizer.tokenize(new_sentence) + eos_token
  input_id = tokenizer.convert_tokens_to_ids(tokens)
  input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]
  input_id = np.array([input_id])
  score = model.predict(input_id)[0][0]

  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [160]:
ex_text = '''
제롬 파월 연방준비제도(Fed) 의장이 “미국이 올리는 최종 금리의 수준이 이전 전망보다 더 높을 수 있다”며 이른바 매파적(통화긴축 선호) 발언을 다시 내놓았다.
이 때문에 미국의 금리 수준을 어느 정도 따라갈 수밖에 없는 우리나라도 기준금리 인상을 놓고 한국은행의 고민이 깊어질 전망이다.
파월은 7일(현지시간) 국회 상원 은행위 청문회에 출석해 “경제 지표가 예상보다 더 강세를 보이고 있다”며 “최종적인 금리 수준은 이전에 전망한 것보다 더 높을 가능성이 크다”고 말했다.
그는 “비록 최근 몇 달간 인플레이션이 완화하고 있으나 인플레이션율을 2% 수준까지 낮추기 위한 과정은 멀고 험난한 길이 될 것”이라고 말했다.
파월 의장은 “물가 상승 압력은 지난 연방공개시장위원회(FOMC) 때 예상했던 것보다 높게 유지되고 있다”며 “물가 안정을 회복하기 위해서는 당분간 제한적인 통화정책 기조 유지가 요구된다”고 말했다.
파월 의장은 “역사적인 사례는 성급하게 정책을 완화하는 것을 경계하고 있다”면서 “연준은 물가 안정 목표를 달성하기 위해 모든 것을 다할 것”이라고 밝혔다.
파월 발언 이후 시장에서는 오는 21∼22일 진행되는 연방공개시장위원회(FOMC) 회의에서 연준이 0.5% 포인트 기준금리를 올리는 ‘빅 스텝’을 단행할 것이라는 관측이 힘을 얻고 있다.
우리나라 한은은 지난달에 부진한 경기 등을 고려해 기준금리를 3.50% 그대로 동결했다. 이에 따라 우리나라 금리는 미국(4.50∼4.75%)보다 1.25%포인트 낮아졌다. 하지만 파월의 발언에 따라 이 기조를 계속 유지하긴 어려울 것으로 보인다.
만약 예상대로 연준이 이달에 ‘빅 스텝’에 나서면 한미 기준금리 차이는 1.75%포인트까지 커진다. 기준금리가 미국보다 크게 낮아지면, 더 높은 수익률을 찾아 외국인 투자 자금이 빠져나가고 원화 가치가 떨어질 위험이 커지게 된다.

'''

In [159]:
sentiment_predict(ex_text)

1/1 [==============================] - 1s 1s/step
99.87% 확률로 부정 리뷰입니다.

